# 📊 SyriaTel Customer Churn Prediction

Welcome to this end-to-end data science project focused on customer churn prediction for **SyriaTel**, a telecommunications company.

## 🧠 Problem Understanding

**Business Objective:**  
SyriaTel wants to proactively identify customers likely to **churn** (stop using their services), so they can intervene and reduce revenue loss.

**Technical Objective:**  
This is a binary classification task — I aim to train a model that accurately predicts whether a customer will churn based on historical data.




In [1]:

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, precision_recall_curve,
                             accuracy_score, precision_score, recall_score, f1_score)
from sklearn.feature_selection import SelectKBest, f_classif

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")


All the necessary Python libraries for data manipulation, visualization, and machine learning have been imported at the start. Their usage will become clear as we move through each stage of the pipeline.
